In [1]:
import polars as pl
import pandas as pd

import numpy as np
import panel as pn
pn.extension('tabulator')
from panel.interact import interact

import hvplot.pandas

import holoviews as hv
from holoviews import opts
hv.extension('bokeh')

## Abro fichero

In [2]:
pl_population = (
    pl.read_parquet("Florida Population Change.parquet")
    .with_columns(pl.col('Deaths').apply(lambda x: x*-1).alias('Death'))
)
pl_population.head()

Year,Births,Deaths,Int_Migration,Local_Migration,Saldo,Death
i64,i64,i64,i64,i64,i64,i64
1990,47494,-32673,10417,55216,80454,32673
1991,197279,-133725,46942,161441,271937,133725
1992,192708,-139084,52015,101294,206933,139084
1993,191930,-143495,54541,104433,207409,143495
1994,192564,-147397,60011,141421,246599,147397


## Creo un pandas dataframe

In [3]:
pd_population = pl_population.to_pandas()

## Creo el interactive

In [4]:
i_population = pd_population.interactive()

## Creo un widget para seleccionar

In [5]:
tipo = pn.widgets.RadioButtonGroup(
    name='Tipos de Datos',
    options=['Saldo','Births','Death','Int_Migration','Local_Migration'],
    button_type='success'
)
tipo

RadioButtonGroup(button_type='success', name='Tipos de Datos', options=['Saldo', 'Births', ...], value='Saldo')

## Creo el pipeline pipe_population a partir del interactive i_population
### En la declaracion estoy involucrando al widget tipo

In [6]:
pipe_population = (
    i_population[['Year',tipo]]    
    .reset_index(drop=True)
)
pipe_population.head()

## Conecto el pipeline pipe_population con un widget
### Y voy a tener en la visualizacion al widget tipo por estar involucrado en la
### definicion del pipeline
## El widget creado se llama population

In [7]:
population = pipe_population.hvplot(x='Year',y=tipo,title='Movimiento Demográfico')
population

## Abro Poblacion.csv
### https://www.census.gov/data/tables/time-series/demo/popest/2020s-national-total.html

In [20]:
pl_poblacion = pl.read_csv('Poblacion_Source.csv', sep=';')

In [21]:
pl_poblacion.head()

state,2020,2021,2022
str,str,str,str
""".Alabama""","""5,031,362""","""5,049,846""","""5,074,296"""
""".Alaska""","""732,923""","""734,182""","""733,583"""
""".Arizona""","""7,179,943""","""7,264,877""","""7,359,197"""
""".Arkansas""","""3,014,195""","""3,028,122""","""3,045,637"""
""".California""","""39,501,653""","""39,142,991""","""39,029,342"""


In [22]:
pl_poblacion = pl_poblacion.with_columns([
    pl.col('2020').apply(lambda x: x.replace(',','')).cast(pl.Int64),
    pl.col('2021').apply(lambda x: x.replace(',','')).cast(pl.Int64),
    pl.col('2022').apply(lambda x: x.replace(',','')).cast(pl.Int64)
])

In [23]:
pl_poblacion.write_csv('population_USA.csv')